### Q1. What is the difference between \__getattr__ and  \__getattribute__?



A key difference between **\__getattr__** and **\__getattribute__** is that **\__getattr__** is only invoked if the attribute wasn't found the usual ways. It's good for implementing a fallback for missing attributes, and is probably the one of two you want.

**\__getattribute__** is invoked before looking at the actual attributes on the object, and so can be tricky to implement correctly. You can end up in infinite recursions very easily.

New-style classes derive from object, old-style classes are those in Python 2.x with no explicit base class. But the distinction between old-style and new-style classes is not the important one when choosing between **\__getattr__** and **\__getattribute__**.


### \__getattr__

Python will call **\__getattr__** method whenever you request an attribute that hasn't already been defined. In the following example my class Count has no **\__getattr__** method. Now in main when I try to access both obj1.mymin and obj1.mymax attributes everything works fine. But when I try to access obj1.mycurrent attribute -- Python gives me AttributeError: 'Count' object has no attribute 'mycurrent'

In [1]:
class Count():
    def __init__(self,mymin,mymax):
        self.mymin=mymin
        self.mymax=mymax

obj1 = Count(1,10)
print(obj1.mymin)
print(obj1.mymax)
print(obj1.mycurrent)

1
10


AttributeError: 'Count' object has no attribute 'mycurrent'

Now my class Count has **\__getattr__** method. Now when I try to access obj1.mycurrent attribute -- python returns me whatever I have implemented in my **\__getattr__** method. In my example whenever I try to call an attribute which doesn't exist, python creates that attribute and sets it to integer value 0.

In [2]:
class Count:
    def __init__(self,mymin,mymax):
        self.mymin=mymin
        self.mymax=mymax    

    def __getattr__(self, item):
        self.__dict__[item]=0
        return 0
88
obj1 = Count(1,10)
print(obj1.mymin)
print(obj1.mymax)
print(obj1.mycurrent1)

1
10
0


### \__getattribute__

Now lets see the **\__getattribute__** method. If you have **\__getattribute__** method in your class, python invokes this method for every attribute regardless whether it exists or not. So why do we need **\__getattribute__** method? One good reason is that you can prevent access to attributes and make them more secure as shown in the following example.

Whenever someone try to access my attributes that starts with substring 'cur' python raises AttributeError exception. Otherwise it returns that attribute.

In [3]:
class Count:

    def __init__(self,mymin,mymax):
        self.mymin=mymin
        self.mymax=mymax
        self.current=None
   
    def __getattribute__(self, item):
        if item.startswith('cur'):
            raise AttributeError
        return object.__getattribute__(self,item) 
        # or you can use ---return super().__getattribute__(item)

obj1 = Count(1,10)
print(obj1.mymin)
print(obj1.mymax)
print(obj1.current)

1
10


AttributeError: 

Important: In order to avoid infinite recursion in **\__getattribute__** method, its implementation should always call the base class method with the same name to access any attributes it needs. For example: object.**\__getattribute__(self, name)** or **super().\__getattribute__(item)** and not **self.\__dict__[item]**

In [4]:
class Count(object):

    def __init__(self,mymin,mymax):
        self.mymin=mymin
        self.mymax=mymax
        self.current=None

    def __getattr__(self, item):
            self.__dict__[item]=0
            return 0

    def __getattribute__(self, item):
        if item.startswith('cur'):
            raise AttributeError
        return object.__getattribute__(self,item)
        # or you can use ---return super().__getattribute__(item)
        # note this class subclass object

obj1 = Count(1,10)
print(obj1.mymin)
print(obj1.mymax)
print(obj1.current)

1
10
0


### Q2. What is the difference between properties and descriptors?

There are several ways that we can tap into Python's internal mechanisms for getting and setting attribute values. The most accessible technique is to use the property function to define get, set and delete methods associated with an attribute name. The property function builds descriptors for you. A slightly less accessible, but more extensible and reusable technique is to define descriptor classes yourself. This allows you considerable flexibility. You do this by creating a class which defines get, set and delete methods, and you associate your descriptor class with an attribute name.

The property function gives us a handy way to implement a simple descriptor without defining a separate class. Rather than create a complete class definition, we can write getter and setter method functions, and then bind these functions to an attribute name.

In [ ]:
### Descriptor example

In [6]:
class Celsius( object ):
    def __init__( self, value=0.0 ):
        self.value= float(value)
    def __get__( self, instance, owner ):
        return self.value
    def __set__( self, instance, value ):
        self.value= float(value)

class Farenheit( object ):
    def __get__( self, instance, owner ):
        return instance.celsius * 9 / 5 + 32
    def __set__( self, instance, value ):
        instance.celsius= (float(value)-32) * 5 / 9

class Temperature( object ):
    celsius= Celsius()
    farenheit= Farenheit()
    
oven= Temperature()
oven.farenheit= 450
oven.celsius

232.22222222222223

In [7]:
oven.celsius= 175
oven.farenheit

347.0

In [8]:
### Propety Example

In [9]:
class Temperature( object ):
    def fget( self ):
        return self.celsius * 9 / 5 + 32
    def fset( self, value ):
        self.celsius= (float(value)-32) * 5 / 9
    farenheit= property( fget, fset )
    def cset( self, value ):
        self.cTemp= float(value)
    def cget( self ):
        return self.cTemp
    celsius= property( cget, cset, doc="Celsius temperature" )

oven= Temperature()
oven.farenheit= 450
oven.celsius



232.22222222222223

In [10]:

oven.celsius= 175
oven.farenheit


347.0

### Q3. What are the key differences in functionality between \__getattr__ and \__getattribute__, as well as properties and descriptors

#### getattr

getattr (object, name[, default])Is one of Python's built-in functions, its role is to get the properties of the object.

    Object object
    Name attribute name
    Default The default value returned when the property does not exist


In [11]:
class Foo:
    def __init__(self, x):
        self.x = x

f = Foo(10)
getattr(f, 'x')
f.x


10

In [12]:
getattr(f, 'y', 'bar')


'bar'

\_ __getattr _ __

object. \__getattr__(self, name)Is an object method that is called if the object's properties are not found.

This method should return the property value or throwAttributeErrorabnormal.

Note that if the object property can be found through the normal mechanism, it will not be called.\__getattr__method.

In [13]:
class Frob:
    def __init__(self, bamf):
        self.bamf = bamf
    def __getattr__(self, name):
        return 'Frob does not have `{}` attribute.'.format(str(name))

f = Frob("bamf")
f.bar


'Frob does not have `bar` attribute.'

In [14]:
f.bamf


'bamf'

\____ getattribute __ __

This method is called unconditionally when accessing the properties of an object. This method only works for new classes.
The new class is a class that integrates from object or type.

If the class is also defined at the same time\__getattr__()Method, it will not be called\__getattr__()Method unless\__getattribute__()Method shows the call\__getattr__()Or thrownAttributeError。

The method should return the property value or throwAttributeErrorabnormal.

To avoid infinite recursion in methods, you should always use the methods of the base class to get the properties:

>>> object.\__getattribute__(self, name).

grammar:object. \__getattribute__(self, name)

In [16]:
class Frob(object):
    def __getattribute__(self, name):
        print("getting `{}`".format(str(name)))
        return object.__getattribute__(self, name)
f = Frob()
f.bamf = 10


In [17]:
f.bamf


getting `bamf`


10

\____ __ get ___ _

**\__get__()** The method is one of the descriptor methods. Descriptors are used to transform access object properties into call descriptor methods.

In [19]:
class Descriptor(object):
    def __get__(self, obj, objtype):
        print("get value=%s" % self.val) 
        return self.val
 
    def __set__(self, obj, val):
        print("set value=%s" % val)
        self.val = val
 
class Stu(object):
    age = Descriptor()
 
stu = Stu()
stu.age = 12    # set value=12
print (stu.age)   # get value=12

set value=12
get value=12
12
